In [1]:
import pandas as pd
import pyodbc

In [3]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-5VCHN8F\\NODESERVER;'
    'DATABASE=chick;'
    'Trusted_Connection=yes;'
)

In [8]:
# Dictionary of queries with file names
queries = {
    "locations_per_state.csv": """
        SELECT state, COUNT(*) AS num_locations
        FROM chickfilaData
        GROUP BY state
        ORDER BY num_locations DESC;
    """,

    "avg_price_per_state.csv": """
        SELECT state, AVG(cost_of_chicken_sandwich) AS avg_price
        FROM chickfilaData
        GROUP BY state
        ORDER BY avg_price DESC;
    """,

    "services_counts.csv": """
        SELECT
            SUM(CASE WHEN Mobile_Orders = 'True' THEN 1 ELSE 0 END) AS mobile_orders,
            SUM(CASE WHEN Catering = 'True' THEN 1 ELSE 0 END) AS catering,
            SUM(CASE WHEN Pickup = 'True' THEN 1 ELSE 0 END) AS pickup,
            SUM(CASE WHEN Delivery = 'True' THEN 1 ELSE 0 END) AS delivery,
            SUM(CASE WHEN WiFi = 'True' THEN 1 ELSE 0 END) AS wifi,
            SUM(CASE WHEN Playground = 'True' THEN 1 ELSE 0 END) AS playground,
            SUM(CASE WHEN Breakfast_Served = 'True' THEN 1 ELSE 0 END) AS breakfast,
            SUM(CASE WHEN Drive_thru = 'True' THEN 1 ELSE 0 END) AS drive_thru
        FROM chickfilaData;
    """,

    "locations_with_all_services.csv": """
        SELECT id, state, location
        FROM chickfilaData
        WHERE Mobile_Orders = 'True'
          AND Catering = 'True'
          AND Pickup = 'True'
          AND Delivery = 'True'
          AND WiFi = 'True'
          AND Playground = 'True'
          AND Breakfast_Served = 'True'
          AND Drive_thru = 'True';
    """,

    "delivery_no_drive_thru.csv": """
        SELECT id, state, location
        FROM chickfilaData
        WHERE Delivery = 'True' AND Drive_thru = 'False';
    """,

    "breakfast_no_playground.csv": """
        SELECT id, state, location
        FROM chickfilaData
        WHERE Breakfast_Served = 'True' AND Playground = 'False';
    """,

    "top10_expensive_sandwiches.csv": """
        SELECT TOP 10 state, location, cost_of_chicken_sandwich
        FROM chickfilaData
        ORDER BY cost_of_chicken_sandwich DESC;
    """,

    "avg_price_wifi_vs_no_wifi.csv": """
        SELECT WiFi, AVG(cost_of_chicken_sandwich) AS avg_price
        FROM chickfilaData
        GROUP BY WiFi;
    """,

    "price_vs_services.csv": """
        SELECT 
           (case when Mobile_Orders='True' then 1 else 0 end+case when Catering='True' then 1 else 0 end 
            +case when Pickup='True' then 1 else 0  end +case when Delivery='True' then 1 else 0 end
            + case when WiFi='True' then 1 else 0 end 
            + case when Playground='True' then 1 else 0 end  + case when Breakfast_served='True' then 1 else 0 end
            +case when Drive_thru='True' then 1 else 0 end ) AS total_services,
            AVG(cost_of_chicken_sandwich) AS avg_price
        FROM chickfilaData
        GROUP BY 
           (case when Mobile_Orders='True' then 1 else 0 end+case when Catering='True' then 1 else 0 end 
            +case when Pickup='True' then 1 else 0  end +case when Delivery='True' then 1 else 0 end
            + case when WiFi='True' then 1 else 0 end 
            + case when Playground='True' then 1 else 0 end  + case when Breakfast_served='True' then 1 else 0 end
            +case when Drive_thru='True' then 1 else 0 end )
        ORDER BY total_services DESC;
    """,

    "delivery_and_drive_thru.csv": """
        SELECT COUNT(*) AS num_locations
        FROM chickfilaData
        WHERE Delivery = 'True' AND Drive_thru = 'True';
    """,

    "breakfast_percentage_per_state.csv": """
        SELECT state,
               COUNT(*) AS total_locations,
               SUM(CASE WHEN Breakfast_Served = 'True' THEN 1 ELSE 0 END) AS breakfast_count,
               (SUM(CASE WHEN Breakfast_Served = 'True' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS breakfast_percentage
        FROM chickfilaData
        GROUP BY state
        ORDER BY breakfast_percentage DESC;
    """,

    "top5_service_combinations.csv": """
        SELECT TOP 5
            Mobile_Orders, Catering, Pickup, Delivery, WiFi, Playground, Breakfast_Served, Drive_thru,
            COUNT(*) AS combo_count
        FROM chickfilaData
        GROUP BY Mobile_Orders, Catering, Pickup, Delivery, WiFi, Playground, Breakfast_Served, Drive_thru
        ORDER BY combo_count DESC;
    """
}

# Execute queries and save CSV
for file_name, query in queries.items():
    df = pd.read_sql(query, conn)
    df.to_csv(file_name, index=False)
    print(f"✅ Saved: {file_name}")

# Close connection
conn.close()

✅ Saved: locations_per_state.csv
✅ Saved: avg_price_per_state.csv
✅ Saved: services_counts.csv
✅ Saved: locations_with_all_services.csv
✅ Saved: delivery_no_drive_thru.csv
✅ Saved: breakfast_no_playground.csv
✅ Saved: top10_expensive_sandwiches.csv
✅ Saved: avg_price_wifi_vs_no_wifi.csv
✅ Saved: price_vs_services.csv
✅ Saved: delivery_and_drive_thru.csv
✅ Saved: breakfast_percentage_per_state.csv
✅ Saved: top5_service_combinations.csv


C:\Users\Ismail\AppData\Local\Temp\ipykernel_17092\635169440.py:113: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
import matplotlib.pyplot as plt
import os

# Create folder for visuals if not exists
os.makedirs("visuals", exist_ok=True)

# Load CSV files
locations_per_state = pd.read_csv("locations_per_state.csv")
avg_price_per_state = pd.read_csv("avg_price_per_state.csv")
services_counts = pd.read_csv("services_counts.csv")
top10_expensive = pd.read_csv("top10_expensive_sandwiches.csv")
price_vs_services = pd.read_csv("price_vs_services.csv")
breakfast_percentage = pd.read_csv("breakfast_percentage_per_state.csv")
top5_combos = pd.read_csv("top5_service_combinations.csv")


# 1. Locations per state
plt.figure(figsize=(12,6))
plt.bar(locations_per_state['state'], locations_per_state['num_locations'])
plt.title("Number of Locations per State")
plt.xlabel("State")
plt.ylabel("Number of Locations")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("visuals/locations_per_state.png")
plt.close()

# 2. Average price per state
plt.figure(figsize=(12,6))
plt.bar(avg_price_per_state['state'], avg_price_per_state['avg_price'])
plt.title("Average Chicken Sandwich Price per State")
plt.xlabel("State")
plt.ylabel("Average Price ($)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("visuals/avg_price_per_state.png")
plt.close()

# 3. Services counts
services_counts.T.plot(
    kind="bar",
    legend=False,
    figsize=(10,6),
    color="skyblue"
)
plt.title("Number of Locations Offering Each Service")
plt.ylabel("Number of Locations")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("visuals/services_counts.png")
plt.close()

# 4. Top 10 expensive sandwiches
plt.figure(figsize=(10,6))
plt.barh(top10_expensive['location'], top10_expensive['cost_of_chicken_sandwich'])
plt.title("Top 10 Most Expensive Chicken Sandwich Locations")
plt.xlabel("Price ($)")
plt.ylabel("Location")
plt.tight_layout()
plt.savefig("visuals/top10_expensive_sandwiches.png")
plt.close()

# 5. Price vs number of services
plt.figure(figsize=(8,6))
plt.plot(price_vs_services['total_services'], price_vs_services['avg_price'], marker='o')
plt.title("Average Price vs Number of Services Offered")
plt.xlabel("Number of Services")
plt.ylabel("Average Price ($)")
plt.grid(True)
plt.tight_layout()
plt.savefig("visuals/price_vs_services.png")
plt.close()

# 6. Breakfast percentage per state
plt.figure(figsize=(12,6))
plt.bar(breakfast_percentage['state'], breakfast_percentage['breakfast_percentage'])
plt.title("Breakfast Availability Percentage per State")
plt.xlabel("State")
plt.ylabel("Breakfast Percentage (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("visuals/breakfast_percentage_per_state.png")
plt.close()

# 7. Top 5 service combinations
plt.figure(figsize=(10,6))
plt.barh(range(len(top5_combos)), top5_combos['combo_count'])
plt.yticks(range(len(top5_combos)), [
    f"Combo {i+1}" for i in range(len(top5_combos))
])
plt.title("Top 5 Service Combinations")
plt.xlabel("Number of Locations")
plt.tight_layout()
plt.savefig("visuals/top5_service_combinations.png")
plt.close()

print("✅ All visuals generated and saved in 'visuals/' folder")


✅ All visuals generated and saved in 'visuals/' folder


In [10]:
# Create folder for extra visuals
os.makedirs("visuals_extra", exist_ok=True)

# Load CSV files
locations_per_state = pd.read_csv("locations_per_state.csv")
avg_price_per_state = pd.read_csv("avg_price_per_state.csv")
services_counts = pd.read_csv("services_counts.csv")
top10_expensive = pd.read_csv("top10_expensive_sandwiches.csv")
price_vs_services = pd.read_csv("price_vs_services.csv")
breakfast_percentage = pd.read_csv("breakfast_percentage_per_state.csv")
top5_combos = pd.read_csv("top5_service_combinations.csv")

# 1. Top 10 States with Most Locations
top10_states = locations_per_state.nlargest(10, 'num_locations')
plt.figure(figsize=(10,6))
plt.bar(top10_states['state'], top10_states['num_locations'], color="orange")
plt.title("Top 10 States with Most Locations")
plt.xlabel("State")
plt.ylabel("Number of Locations")
plt.tight_layout()
plt.savefig("visuals_extra/top10_states_locations.png")
plt.close()

# 2. Distribution of Locations (Pie Chart)
plt.figure(figsize=(8,8))
plt.pie(locations_per_state['num_locations'], labels=locations_per_state['state'], autopct='%1.1f%%')
plt.title("Distribution of Locations by State")
plt.savefig("visuals_extra/locations_distribution_pie.png")
plt.close()

# 3. Top 10 Most Expensive States (Average Price)
top10_exp_states = avg_price_per_state.nlargest(10, 'avg_price')
plt.figure(figsize=(10,6))
plt.bar(top10_exp_states['state'], top10_exp_states['avg_price'], color="red")
plt.title("Top 10 States with Highest Average Sandwich Price")
plt.xlabel("State")
plt.ylabel("Average Price ($)")
plt.tight_layout()
plt.savefig("visuals_extra/top10_expensive_states.png")
plt.close()

# 4. Histogram of Average Prices
plt.figure(figsize=(8,6))
plt.hist(avg_price_per_state['avg_price'], bins=10, color="green", edgecolor="black")
plt.title("Distribution of Average Prices Across States")
plt.xlabel("Average Price ($)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig("visuals_extra/avg_price_histogram.png")
plt.close()

# 5. Services Popularity (Pie Chart)
services_sums = services_counts.sum()
plt.figure(figsize=(8,8))
plt.pie(services_sums, labels=services_sums.index, autopct='%1.1f%%')
plt.title("Services Popularity Percentage")
plt.savefig("visuals_extra/services_popularity_pie.png")
plt.close()

# 6. Boxplot of Expensive Sandwich Prices
plt.figure(figsize=(8,6))
plt.boxplot(top10_expensive['cost_of_chicken_sandwich'])
plt.title("Boxplot of Top 10 Expensive Sandwich Prices")
plt.ylabel("Price ($)")
plt.savefig("visuals_extra/expensive_sandwiches_boxplot.png")
plt.close()

# 7. Scatter Plot (Services vs Price)
plt.figure(figsize=(8,6))
plt.scatter(price_vs_services['total_services'], price_vs_services['avg_price'], c="blue", alpha=0.7)
plt.title("Scatter Plot: Number of Services vs Average Price")
plt.xlabel("Number of Services")
plt.ylabel("Average Price ($)")
plt.grid(True)
plt.tight_layout()
plt.savefig("visuals_extra/scatter_services_vs_price.png")
plt.close()

# 8. Top 5 States Serving Breakfast
top5_breakfast = breakfast_percentage.nlargest(5, 'breakfast_percentage')
plt.figure(figsize=(10,6))
plt.bar(top5_breakfast['state'], top5_breakfast['breakfast_percentage'], color="purple")
plt.title("Top 5 States Serving Breakfast")
plt.xlabel("State")
plt.ylabel("Breakfast Percentage (%)")
plt.tight_layout()
plt.savefig("visuals_extra/top5_breakfast_states.png")
plt.close()

# 9. Service Combination Share (Pie Chart)
plt.figure(figsize=(8,8))
plt.pie(top5_combos['combo_count'], labels=[f"Combo {i+1}" for i in range(len(top5_combos))], autopct='%1.1f%%')
plt.title("Top 5 Service Combinations Share")
plt.savefig("visuals_extra/service_combinations_share.png")
plt.close()

print("✅ Extra visuals generated and saved in 'visuals_extra/' folder")


✅ Extra visuals generated and saved in 'visuals_extra/' folder


In [16]:
from fpdf import FPDF

# ---------- PDF Setup ----------
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.set_font("Arial", '', 12)

# ---------- Helper Function ----------
def add_chart_section(title, description, image_path=None):
    # استبدال dash الطويل بالقصير لتجنب مشاكل الترميز
    title = title.replace("–", "-")
    description = description.replace("–", "-")
    
    pdf.set_font("Arial", 'B', 14)
    pdf.multi_cell(0, 7, title)
    pdf.ln(2)
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 6, description)
    pdf.ln(3)
    if image_path and os.path.exists(image_path):
        pdf.image(image_path, w=180)
        pdf.ln(5)

# ---------- Cover Page ----------
pdf.add_page()
pdf.set_font("Arial", 'B', 18)
pdf.multi_cell(0, 10, "Restaurant Network Analysis - Chicken Sandwich Insights", align="C")
pdf.ln(10)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 6, "Prepared by: Data Analyst")
pdf.multi_cell(0, 6, "Date: September 2025")
pdf.ln(10)
pdf.multi_cell(0, 6, "This report provides a comprehensive analysis of a restaurant network offering chicken sandwiches across multiple U.S. states. It includes pricing, service availability, location distribution, and patterns in service combinations.")

# ---------- Section 1: Main Charts ----------
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.multi_cell(0, 8, "1. Main Charts")
pdf.ln(5)

main_charts = [
    ("Average Chicken Sandwich Price per State",
     "Displays average chicken sandwich prices across states, showing a general decreasing trend. NY and DC exhibit the highest prices (~$6.2), while states like AZ and NH are lowest (~$4.8-$4.9).",
     "visuals/avg_price_per_state.png"),

    ("Breakfast Availability Percentage per State",
     "Illustrates the proportion of locations offering breakfast per state. CA, TX, NY lead with 89-82%, while WA and FL are slightly lower (78-80%).",
     "visuals/breakfast_percentage_per_state.png"),

    ("Number of Locations per State",
     "Shows location distribution across states. TX has 470 locations, GA 260, FL 245. Smallest states have <100 locations.",
     "visuals/locations_per_state.png"),

    ("Average Price vs Number of Services Offered",
     "Examines correlation between services and sandwich price. Slight downward trend; more services correlate with marginally lower prices (5.4-5.8 $ down to ~5.0 $).",
     "visuals/price_vs_services.png"),

    ("Number of Locations Offering Each Service",
     "Breakfast (~2500 locations) and Mobile Orders (~2200) are most common. Playground and Drive-thru are least (~200-600).",
     "visuals/services_counts.png"),

    ("Top 5 Service Combinations",
     "Combo 1 dominates with ~1400 locations; subsequent combos have significantly fewer locations.",
     "visuals/top5_service_combinations.png"),

    ("Top 10 Most Expensive Chicken Sandwich Locations",
     "Prices range 9-11 $. Highest in stadiums and high-traffic areas.",
     "visuals/top10_expensive_sandwiches.png"),
]

for title, desc, img in main_charts:
    add_chart_section(title, desc, img)

# ---------- Section 2: Additional Charts ----------
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.multi_cell(0, 8, "2. Additional Charts")
pdf.ln(5)

additional_charts = [
    ("Top 5 States Serving Breakfast",
     "Shows top 5 states with highest breakfast availability: CA 89%, TX 85%, NY 82%, FL 80%, WA 78%.",
     "visuals_extra/top5_breakfast_states.png"),

    ("Top 10 States with Highest Average Sandwich Price",
     "NY 6.20, DC 5.90, WA 5.80, MA 5.78, CA 5.75, OR 5.70, RI 5.70, CT 5.65, NH 5.50, AZ 5.45.",
     "visuals_extra/top10_expensive_states.png"),

    ("Top 10 States with Most Locations",
     "TX 470, GA 260, FL 245, MA 190, CA 175, OR 140, RI 100, CT 100, NH 100, AZ 95.",
     "visuals_extra/top10_states_locations.png"),

    ("Distribution of Average Prices Across States",
     "Histogram shows clustering between $5-6 with small variations across states.",
     "visuals_extra/avg_price_histogram.png"),

    ("Boxplot of Top 10 Expensive Sandwich Prices",
     "Quartiles: Min ~6.7, Q1 6.75, Median 9.4, Q3 9.9, Max 11 $.",
     "visuals_extra/expensive_sandwiches_boxplot.png"),

    ("Distribution of Locations by State",
     "Pie chart illustrating market concentration across states.",
     "visuals_extra/locations_distribution_pie.png"),

    ("Scatter Plot: Number of Services vs Average Price",
     "Shows slight negative correlation; more services marginally reduce price (5.02-5.8 $).",
     "visuals_extra/scatter_services_vs_price.png"),

    ("Top 5 Service Combinations Share",
     "Pie chart: Combo 1 58.8%, Combo 2 18.5%, Combo 3 8.8%, Combo 4 7.8%, Combo 5 6.1%.",
     "visuals_extra/service_combinations_share.png"),

    ("Services Popularity Percentage",
     "Pie chart: Breakfast 14%, Mobile Orders 13.7%, Pickup 13.5%, Catering 13.5%, Wi-Fi 12.8%, Drive-thru 11.8%, Playground 8.9%.",
     "visuals_extra/services_popularity_pie.png"),
]

for title, desc, img in additional_charts:
    add_chart_section(title, desc, img)

# ---------- Section 3: Key Findings & Recommendations ----------
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.multi_cell(0, 8, "3. Key Findings & Recommendations")
pdf.ln(5)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 6, """Key Findings:
- Price disparities exist across states; NY highest, AZ lowest.
- Breakfast and Mobile Orders are core services; Playground and Drive-thru less common.
- TX dominates in number of locations; other states have <100.
- Popular service combinations indicate operational efficiencies.
- Slight negative correlation between number of services and average price.

Recommendations:
- Adjust pricing in high-cost states; consider promotions in low-price states.
- Expand less common services in high-demand regions.
- Target states with fewer locations for expansion opportunities.
- Focus on popular service combinations to optimize operations.
- Maximize breakfast promotions in top states, incentivize in lower-percentage states.""")
pdf.ln(5)

# ---------- Save PDF ----------
pdf.output("Chicken_Sandwich_Network_Report.pdf")
print("✅ PDF report generated: Chicken_Sandwich_Network_Report.pdf")

✅ PDF report generated: Chicken_Sandwich_Network_Report.pdf
